# <span style="color:#F72585">Redes GRU (Gated Recurrent Unit)</span>

## <span style="color:#4361EE">Introducción</span>



Una red neuronal recurrente (RNR) es una extensión de una red neuronal convencional,
que es capaz de manejar una entrada de secuencia de longitud variable. La RNR maneja la longitud variable de las secuencias mediante un estado oculto recurrente cuya activación en cada momento depende del estado anterior. 

Más formalmente, dada una secuencia $\mathbf{x}= (x_1, x_2, \ldots, x_T)$, la RNR actualiza su estado oculto recurrente $h_t$ mediante 


$$
h_t = \begin{cases}
0,  &\text{ si } t=0,\\
\phi(h_{t-1},x_t), &\text{en otro caso}.
\end{cases}
$$


Si $g$ es una función de activación  suave, como un sigmoide o una tangente hiperbólica, es común definir

$$
h_t = g(Wh_{t-1} + Ux_t+ b)
$$

Una RNR generativa genera una distribución de probabilidad sobre el siguiente elemento de la secuencia, dado su estado actual $h_t$, y este modelo generativo puede capturar una distribución sobre secuencias de longitud variable mediante el uso de un símbolo de salida especial para representar el final de la secuencia. La secuencia la probabilidad se puede descomponer en

$$
p (x_1,\ldots, x_T) = p (x_1) p (x_2 | x_1) p (x_3 | x_1, x_2) \ldots p (x_T | x_1,\ldots, x_{T − 1}), 
$$

donde el último elemento es un valor especial de final de secuencia. Modelamos cada probabilidad condicional distribución con

$$
p (x_t | x_1,\ldots, x_{t − 1}) = g (h_t)
$$

El problema con este modelo, es que el cálculo del gradiente tiende a volverse cero o a explotar. 

Dos líneas de trabajo impulsó esta situación. Por un lado se inició la búsqueda de nuevas técnicas para para el uso del gradiente en el proceso de optimización de  la función de costo y por el otro, el desarrollo de nuevos modelos de redes neuronales. 


La primera línea ha producido nuevas técnicas de optimización estocástica basados en el gradiente, que han sido usado exitosamente en redes generales. 

La segunda línea llevó al desarrollo de las redes [LSTM](Intro_LSTM.ipynb), en las cuales la función de activación consiste en una transformación afinada seguida por una simple no linealidad de elementos mediante el uso de unidades de compuerta, [Hochreiter y Schmidhuber, 1997](https://www.bioinf.jku.at/publications/older/2604.pdf). 


Más recientemente, otro tipo de unidad recurrente, a la que nos referimos como una unidad recurrente cerrada (GRU), fue propuesta por [Cho et al. 2014](https://arxiv.org/pdf/1409.1259.pdf). Puede consultar una comparación entre LSTM y GRU en [Junyoung Chung et al., 2014](https://arxiv.org/pdf/1412.3555v1.pdf). De estas unidades recurrentes se ha demostrado que funcionan bien en tareas que requieren captura de dependencias a largo plazo.  Esas tareas incluyen, pero no se limitan a reconocimiento de voz, música,...

## <span style="color:#4361EE">Funcionamiento de una red GRU</span>

Una red de unidad recurrente cerrada (GRU) permite que cada unidad recurrente capture de forma adaptativa dependencias de diferentes escalas de tiempo. De manera similar a la unidad LSTM, la GRU tiene puertas que modulan el flujo de información dentro de la unidad.  Sin embargo, a diferencia de las redes LSTM no tiene celdas de memoria separadas. Las dos  imágenes muestran la estructura general de las compuertas GRU.


Estos diagramas muestran la estructura matemática dentro de la compuerta en el tiempo $t$, con reinicio antes o después de de la multiplicación.


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/compuerta_GRU_after_false.jpg" width="800" height="400" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

reset_after: convención GRU (si aplicar la puerta de reinicio después o antes de la multiplicación de matrices). False = "antes", True = "después" (este es el caso por defecto).


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/compuerta_GRU_after_true.jpg" width="800" height="400" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

La notación que usamos es bastante estándar. 

1. $+$ : indica suma de vectores
2. $\sigma$ : representa a la función de activación sigmoide
3. $\tanh$ : representa a la función de activación tangente hiperbólica.
4.$\odot$ : es producto componente a componente (producto de Hamard)

Una GRU tien dos tipos de puerta: actualización (update) y reinicio (reset).

## <span style="color:#4361EE">Puerta de actualización-update</span>


Comenzamos calculando la puerta de actualización $z_t$ para el paso de tiempo $t$ usando la fórmula:

$$
z_t = \sigma(W_z x_t + U_zh_{t-1} + b_z)
$$

en donde $W_z$ y $U_z$ son pesos asociados $x_t$ (la nueva entrada) y $h_{t-1}$, (la  información procedente de la unidad anterior).


La puerta de actualización ayuda al modelo a determinar qué cantidad de la información pasada (de los pasos de tiempo anteriores) debe transmitirse al futuro, combinandola  con la nueva información. 


Eso es realmente poderoso porque el modelo puede decidir copiar toda la información del pasado y eliminar el riesgo de desvanecer del gradiente.

## <span style="color:#4361EE">Puerta reinicio</span>

Esencialmente, esta puerta se utiliza para que el modelo decida qué cantidad de información pasada debe olvidar. Para calcularla, utilizamos:

$$
r_t = \sigma(W_r x_t + U_rh_{t-1} + b_r),
$$

en donde $W_r$ y $U_r$ son pesos asociados $x_t$ (la nueva entrada) y $h_{t-1}$, (la  información procedente de la unidad anterior).

## <span style="color:#4361EE">Activación candidata</span>



Veamos cómo afectarán exactamente las puertas al resultado final. Primero, comenzamos con el uso de la puerta de reinicio. Introducimos un nuevo contenido de memoria que utilizará la puerta de reinicio para almacenar la información relevante del pasado. Se calcula de la siguiente manera:


$$
\tilde{h}_t = \tanh(W_nx_t + U_n(r_t\odot h_{t-1})+b_n),
$$

en donde $W_n$ y $U_n$ son pesos asociados a las  $x$'s y a las $h$'s respectivamente.

## <span style="color:#4361EE">Actualización del estado recurrente</span>

La activación de la GRU en tiempo $t$ es una interpolación lineal entre la activación previa ${h}_{t-1}$ y la activación candidata $\tilde{h}_t$ definida arriba. 

En símbolos tenemos:


$$
h_t = (1-z_t)\odot h_{t-1} + z_t \odot \tilde{h}_t 
$$

La siguiente imagen ilustra la arquitectura de la red.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Redes_Recurrentes/Imagenes/plano_gru.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

## <span style="color:#4361EE">Resumen de la matemática en la red GRU</span>

$$
z_t = \sigma(W_z x_t + U_zh_{t-1} + b_z)
$$

$$
r_t = \sigma(W_r x_t + U_rh_{t-1} + b_r),
$$

$$
\tilde{h}_t = \tanh(W_nx_t + U_n(r_t\odot h_{t-1})+b_n),
$$

$$
h_t = (1-z_t)\odot h_{t-1} + z_t \odot \tilde{h}_t 
$$

## <span style="color:#4361EE">Computación  en una capa GRU de Keras</span>

Cualquier capa de Keras siempre espera un batch de datos. En el caso de una capa LSTM Keras espera tensores 3D de la siguiente forma

+ [batch_size, time_step, feature]



Por ejemplo un tensor de entrada de tamaño [32, 10, 8], la capa Keras lo interpreta como
- batch_size = 32, es decir 32 ejemplos.
- time_step = 10, es decir secuencias de entrada de tamaño 10. Por ejemplo en una serie de tiempo este es el tamaño de la ventana de entrada.
- feature = 8, es decir la variable de entrada es de tamaño 8. Por ejemplo, en series de timepo univariadas, feature = 1. En una serie multivariada con 8 variables, features = 8. En modelos de lenguaje natural feature = tamaño de representación de acada token. Usualmente correspondería al tamaño del embedding.

La salida de la capa corresponde al tamaño del estado oculto. Por ejemplo, si el estado oculto tiene tamaño, la salida de la capa es de tamaño [batch_size, 4].

### <span style="color:#4CC9F0">Ejemplo</span>


In [34]:
import tensorflow as tf
inputs = tf.random.normal([32,10,8])
gru = tf.keras.layers.GRU(4) # gru es una capa de tamaño de salidad 4.
output = gru(inputs)
print (output.shape)


(32, 4)


### <span style="color:#4CC9F0">Recibiendo toda la secuencia del valor del estado oculto</span>

En algunos casos es necesario disponer del valor del estado oculto para cada valor en la secuenci de entrada. Esta secuencia tiene tamaño [batch_size, time_step, output_size]
En el sugiente ejemplo se tiene que

- `return_sequences` son todos lo estados del estado oculto
- `return_state` es el último valor del estado oculto

In [35]:
gru = tf.keras.layers.GRU(4, return_sequences = True, return_state=True)
whole_seq_output, final_memory_state = gru(inputs)

In [36]:
print(whole_seq_output.shape)
print(final_memory_state.shape)


(32, 10, 4)
(32, 4)


In [37]:
whole_seq_output[:,-1] == final_memory_state

<tf.Tensor: shape=(32, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True, 

### <span style="color:#4CC9F0">Extracción de los pesos de la capa</span>


In [38]:
help(gru)

Help on GRU in module keras.layers.rnn.gru object:

class GRU(keras.layers.rnn.dropout_rnn_cell_mixin.DropoutRNNCellMixin, keras.layers.rnn.base_rnn.RNN, keras.engine.base_layer.BaseRandomLayer)
 |  GRU(units, activation='tanh', recurrent_activation='sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False, time_major=False, reset_after=True, **kwargs)
 |  
 |  Gated Recurrent Unit - Cho et al. 2014.
 |  
 |  See [the Keras RNN API guide](https://www.tensorflow.org/guide/keras/rnn)
 |  for details about the usage of RNN API.
 |  
 |  Based on available runtime hardware and constraints, this layer
 |  will choose differen

In [39]:
weights = gru.get_weights()

In [40]:
len(weights)

3

In [41]:
weights[0].shape

(8, 12)

In [42]:
weights[1].shape

(4, 12)

In [43]:
weights[2].shape

(2, 12)

Note que los pesos de la capa GRU Están organizados de la siguiente forma

In [44]:
units = 4 # tamaño del estado oculto
W = gru.get_weights()[0]
W_z = W[:,:units]
W_r = W[:, units:units*2]
W_n = W[:, units*2:units*3]


U = gru.get_weights()[1]
U_z = W[:,:units]
U_r= W[:, units:units*2]
U_n = W[:, units*2:units*3]

b = gru.get_weights()[2] 
b_z = b[:units]
b_r = b[ units:units*2]
b_n = b[units*2:units*3]


## <span style="color:#4361EE">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
1. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
1. [Dive into Deep Learnig](https://d2l.ai/)
1. [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
1. Ralf C. Staudemeyer and Eric Rothstein Morris,[*Understanding LSTM a tutorial into Long Short-Term Memory Recurrent Neural Networks*](https://arxiv.org/pdf/1909.09586.pdf), arxiv, September 2019
1. Karpathy, [*The Unreasonable Effectiveness of Recurrent Neural Networks*](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
1.  Cho et al. 2014 [On the Properties of Neural Machine Translation: Encoder–Decoder
Approaches](https://arxiv.org/pdf/1409.1259.pdf)
1. J. Chung, C. Gulcehre, K. Cho, Y. Bengio, [Empirical Evaluation of Gated Recurrent Neural Networks on Sequence  Modeling](https://arxiv.org/pdf/1412.3555v1.pdf)